In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
import argparse
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
test_df = pd.read_csv('../test.csv')
test_df

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [3]:
model_id = 'beomi/KoAlpaca-Polyglot-5.8B'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}) # 모델을 쪼개서 load함
model = prepare_model_for_kbit_training(model)
adapter_name = "koalpaca_finetuned_preprocess" # Adapter의 폴더 명을 가져와야 함
# model에 아까 학습한 adapter를 연결시킴
model = PeftModel.from_pretrained(model, adapter_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


bin C:\Users\user\anaconda3\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [4]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

In [5]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
# model.config.use_cache = False

In [6]:
pipe = pipeline(task="text-generation", # 지정된 task에 맞게 model이 돌아감
                model=model, # 아까 학습시킨 adapter에 모델에 연결된 상태
                tokenizer=tokenizer,
                max_length = 600,
                do_sample=True,
                early_stopping=True,
                temperature=0.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                top_k=1,
                top_p=0.99,
                repetition_penalty = 1.3,
                framework='pt'
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [7]:
def ask(x, is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 답변:"
    )
    # print(ans[0]['generated_text'])
    return ans[0]['generated_text'].split('### 답변:')[1].strip()

In [8]:
answer = ask('방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?')
print(answer)

방청페인트를 사용하려면 몇 가지 특성을 고려해야 합니다. 먼저, 방청페인트는 도장 후 경화되기 전까지 표면이 건조하고 유지될 수 있어야 하며, 이로 인해 도막의 두께가 두꺼워지거나 균일해지지 않을 수 있습니다. 따라서 이러한 특징으로 인해 방청페인트는 주로 외부 환경이나 온도 변화 등에 노출되는 장소에서 사용됩니다. 또한, 방청페인트는 일반적으로 수성 타입의 제품입니다. 물과 같은 용매를 기반으로 만들어지므로 매우 유연하여 쉽게 덧칠할 수 있지만, 내수성 및 내후성이 강하여 특정 조건에서도 잘 견딜 수 있다는 점도 염두에 두시기 바랍니다. 마지막으로, 방청페인트는 철재 구조물 또는 금속 부품의 부식 방지와 보호에도 효과적인 역할을 수행합니다. 이를 통해 건물 내부나 외부의 안전성과 수명을 향상시키는데 도움을 줄 수 있습니다.


In [9]:
answer = ask('원목사이딩을 사용하는 것에 어떤 단점이 있을까요?')
print(answer)

원목사이딩의 장점은 가격대가 저렴하고 관리가 쉽다는 점입니다. 또한 자연스러운 나뭇결로 외관을 연출하여 고급스러움과 내구성 측면에서도 좋은 선택으로 평가됩니다. 하지만 습기에 취약한 특성 때문에 설치나 유지보수에 다소 많은 비용이 소요될 수 있습니다. 이 외에도 환경 친화적인 재료를 사용함으로써 주변 환경과 조화롭게 어울리며, 단열성능 역시 우수합니다. 그러나 외부 충격이나 습기 등으로부터 보호받기 어려운 부분이 있으므로 안전상의 문제도 고려해야 합니다. 추가적으로 설치 및 보수 작업 시 전문 기술이 필요할 수 있어 시간과 노력이 많이 드린다는 점도 염두에 두어야 합니다. 따라서 이러한 장단점들을 종합해보면, 원목사이딩을 선택할 때에는 신중한 판단이 요구된다고 할 수 있겠죠.


In [10]:
test_df.loc[0, "질문"]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'

In [11]:
answer = ask(test_df.loc[0, "질문"])
print(answer)

방청페인트를 시공할 경우 친유성으로 구성되어 있어서 윤활 기능은 하지만 내수성이 약해서 주로 옥외에서 사용해야 하며 특히 금속 표면이나 노출된 마감재와 함께 사용하면 부식을 가속화시킬 수 있습니다. 따라서 녹방지용 방청제나 내구성 강한 방수 처리가 필요합니다. 이러한 특성 때문에 방청페인트를 사용할 때는 적절한 보호장비를 착용하고, 전문가의 조언과 지시를 따르는 것이 중요합니다. 또한, 방청페인트를 사용하여 외부 환경으로부터 보호하기 위해서는 방청 도료를 선택하고 적용하는 방법도 고려해 볼 만 합니다.


In [12]:
test_df.loc[4, "질문"]

'도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?'

In [13]:
answer = ask(test_df.loc[4, "질문"])
print(answer)

도배 후 완전한 건조는 제조사 권장 온도와 습도에 따라 다릅니다. 일반적으로 제조사에서 권장하는 적정온도는 20도~30도가며, 습도가 60% 이상일 때까지 기다려주세요. 만약 제조사의 권장사항을 따르지 않는다면, 시공사에게 연락하여 추가적인 조치 및 확인이 필요할 수 있습니다. 따라서 충분한 시간을 가지고 기다리시면 안전하고 정확한 방법으로 벽지를 처리할 수 있습니다. 만약 제조사나 시공업체로부터 아직 완전히 건조되지 않은 상태라면 즉시 벽지를 제거하고 교환해야 합니다. 이 경우에도 제조사의 지침을 따라야 하므로 반드시 전문가의 조언과 지시를 준수해야 합니다.


### model inference

In [15]:
preds = []
for question in tqdm(test_df['질문'], total=len(test_df)):
    questions = re.split(r'(?<=[?.])', question)
    questions = [q for q in questions if q.strip()]
    print(questions)
    answer = ""
    for q in questions:
        if q and len(q) > 1:
            q = q.strip()
            gen = ask(q)
            print(gen)
            answer += gen + " "
    preds.append(answer.strip())

  0%|          | 0/130 [00:00<?, ?it/s]

['방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?', ' 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?']
방청페인트를 사용하려면 몇 가지 특성을 고려해야 합니다. 먼저, 방청페인트는 도장 후 경화되기 전까지 표면이 건조하고 유지될 수 있어야 하며, 이로 인해 도막의 두께가 두꺼워지거나 균일해지지 않을 수 있습니다. 따라서 이러한 특징으로 인해 방청페인트는 주로 외부 환경이나 온도 변화 등에 노출되는 장소에서 사용됩니다. 또한, 방청페인트는 일반적으로 수성 타입의 제품입니다. 물과 같은 용매를 기반으로 만들어지므로 매우 유연하여 쉽게 덧칠할 수 있지만, 내수성 및 내후성이 강하여 특정 조건에서도 잘 견딜 수 있다는 점도 염두에 두시기 바랍니다. 마지막으로, 방청페인트는 철재 구조물 또는 금속 부품의 부식 방지와 보호에도 효과적인 역할을 수행합니다. 이를 통해 건물 내부나 외부의 안전성과 수명을 향상시키는데 도움을 줄 수 있습니다.
원목사이딩의 장점은 가격대가 저렴하고 시공 및 유지보수가 용이하며 자연스러운 나뭇결로 미적인 측면에서도 좋다는 점입니다. 반면에 단점으로는 습기에 취약하여 변형이나 손상될 수 있고, 곰팡이와 같은 해로운 물질로부터 보호되어야 한다는 점이 있습니다. 이러한 이유로 원목사이딩을 선택할 때는 주변 환경과 상황 등을 고려하여 적절한 재료를 선정해야 합니다.
['도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?']
도배지 안쪽의 철착망 형태의 베어링이 오랫동안 사용되면 녹이 슬게 되어, 이염될 수 있습니다. 따라서 이러한 현상을 방지하기 위해서는 적절한 관리가 필요합니다. 먼저, 온도와 습도를 적절히 유지하고 녹이 생긴 부위에 윤활제나 도포재 등을 발라주어야 합니다. 또한, 부식된 부분을 제거하여 청결 상태를 유지해야 하며, 녹이 없도록 교체 작업이나 수리 작업 시 전문가에게 상담받는 것이 좋습니다. 만약 이러한 조치들이 부적절할 경우 곰팡이가 번식할 가능성이 높아질 수도 있으므로 주의해야 합니다.

피스 하자는 벽지 안쪽에 박혀있는 못이나 기타 다른 작은 물질로 인해 발생하는 하자를 말합니다. 이러한 현상이 일어나는 원인과 책임소재, 그리고 해결 방법 등을 알아보겠습니다. 1) 못자국 : 도배지의 표면에 나타나거나 또는 도배 후에도 발견될 수 있는데 이는 못이 벽면 내에서 부식되어 녹이 슬고 찌꺼기가 쌓이기 때문입니다. - 책임 소재 : 시공자 - 해결 방법 : 부분 혹은 전체 재시공 - 문제점 : 시간이 지남에 따라 쉽게 커지며 건조시간 부족으로 인한 곰팡이 발생 가능성도 존재합니다. 2) 핏자국 : 도배지에 붉은 색이나 검정색의 얼룩무늬 형태로 찍히는 것을 의미합니다. - 책임 소재 : 개인피용 업체 - 해결 방법 : 시공사인테리어업체 - 문제점 : 건강 문제 및 주거 환경에 심각한 영향을 미칠 수 있으며, 관련 법규와 규정에 부합한지 확인해야 합니다. 3) 분진흐름보풀 : 도배지위에 먼지나 특정 장소로부터 떨어져 나오는 모래먼지 같은 분진이 도배지 위에 생기는 현상을 의미합니다. - 책임 소재 : 공간 내부의 습도 조절 불량으로 발생할 수 있습니다. - 해결 방법 : 보수작업 후 재발 방지를 위해 습기 관리가 필요합니다. 4) 접착불량 : 도배지의 접착제가 제대로 부착되었는지 확인하고, 누락된 부분에 추가 작업이 필요하면 전문가에게 상담하여야 합니다. - 책임 소재 : 시공자 - 해결 방법 : 제조사 검증받은 친환경 제품을 사용하여 품질관리를 해야 합니다. 5) 석고보드 누수 : 도배지 속에 들어 있는 석고보드가 누수가 되면 바로 수리해야 하고, 누수 상태가 지속된다면 해당 부분을 교체하여야 합니다. - 책임 소재 : 건물 소유자 - 해결 방법 : 거주자가 물허덕임을 알았지만 무시했다면 소방 시설을 가동하여 자체 소방서에서 구조활동을 할 수 없으므로 먼저 방수용 실리콘 보강공사를 한 뒤 제습기 설치 및 환기를 통해 실내 적정 온도와 습도를 유지시켜야 합니다. 6) 치수염 난방 : 도배 풀의 건조가 충분히 이루어지지 않아 생긴 도배지의 팽창현상으로 인

페인트하도재 처리 없이 바로 칠하는 것은 공간 내에서의 결로 및 습도 증가와 같은 문제들을 유발시킬 수 있습니다. 이러한 상황이 계속된다면 건물 내부 벽면에 부식이나 손상 등의 문제점이 발생할 가능성이 높아집니다. 따라서, 페인트도료를 시공하기 전에 하도재를 통해 도포하고 일정 기간 동안 냉각시킨 후 사용하는 것이 좋으며, 이후에도 적절한 보호제나 건조제를 이용하여 관리해야 합니다. 또한, 외부 환경 조건과 건축물의 구조물 및 재료물 간의 상호작용에 대한 이해와 평가가 필요합니다. 이를 바탕으로 적합한 하도재를 선택하고 적용함으로써 안전하고 효과적인 마감작업을 수행할 수 있습니다.
['바닥재가 남으면 어떻게 처리하는 게 좋을까요?', ' 그리고 장판이 남을 때 어떻게 처리해야 하나요?']
나무재인 티타늄은 고온에 강하고 재활용이 가능하여 저렴한 가격으로 구할 수 있습니다. 또한, 시멘트는 유해물질 배출과 환경호르몬 발생을 억제하고 천연 소재로 인체 무해하며 오랜 기간 사용할 수 있다는 장점이 있지만, 질감이나 마감재의 내구성 측면에서는 다소 약합니다. 추가적으로 선택할 만한 대안으로는 친환경 재질인 유리나 목재 등이 있으며, 이들도 비교적 안전하고 재생 가능한 재료로서 인기를 얻고 있습니다. 다만, 지역 또는 제조업체마다 품질 및 설치 방법에 차이가 있으므로 구매 시 주의해야 합니다. 만약 바닥재를 교체하지 않는다면, 건강 문제와 더불어 주변 가구와 다른 가구나 전자제품에도 영향을 미칠 수 있기 때문에 적절한 시기에 교체하는 것이 중요합니다.
일반적으로 장판의 일부는 쓰레기 봉투에 담아서 버리면 됩니다. 하지만, 이 경우에는 환경에도 고려되어야 합니다. 예를 들어, 폐기물 스티커나 배출방법 등을 통해 재활용 가능한 재료로 분리할 수 있습니다. 또한, 해당 지역의 구청이나 주민센터에서 관련 규정과 지침을 확인하여 적절하게 폐기 처리해 주실 것을 권장드립니다.
['도배지에 생긴 반점을 없애기 위해 가장 효과적인 방법은 무엇인가요?']
도배지 안쪽의 반점과 같은 얼

인테리어 소품을 활용하려면 몇 가지 팁이 필요합니다. 먼저, 액자나 장식품은 공간을 아름답게 만들어주는데 도움이 됩니다. 또한, 자연광 조명과 식물들로 인해 생기 넘치고 밝은 분위기가 조성됩니다. 마지막으로, 색상 팔레트와 질감 소재를 신중하게 고려하여 조화로운 느낌을 주어야 합니다. 이러한 점들이 인테리어 소품을 선택하고 배치함으로써 공간을 더 매력적으로 꾸밀 수 있습니다. 더불어 각각의 아이템이 어울리며, 전체적인 통일감과 생동감을 부여해 줄 것입니다. 따라서, 적절한 디자인 및 색감 조합을 신중히 고민하여 소품을 선택하면 좋은 효과를 얻을 수 있습니다.
['초배지만 남은 벽에 페인트를 칠하면 어떤 문제가 발생하나요?', ' 또한, 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.']
일반적으로 초배지는 매우 얇기 때문에 벽면을 평평하게 유지할 수 있습니다. 따라서 이음부나 틈새가 없으므로페인트를 바로 덧칠해도 표면이 평단해보일 수 있습니다. 또한, 초배지의 경우 기름 또는 용제 성분이 많이 함유되어 있어서 페인트와 결합해서 막을 형성시키거나 얼룩질을 유발할 수도 있습니다.따라서 페인트 작업 시에는 적합한 마감재로 마무리해야 합니다.
속건성 유성 발수제로는 아크릴 에멀젼계 혼화방식을 이용하여 수분은 차단하고 유막만 형성시켜 물이 침투하지 못하도록 보호합니다. 이러한 특징으로 인해 외부 벽면이나 콘크리트 및 금속 등 다양한 표면에서 쉽게 적용할 수 있습니다. 이를 통해 건물 내부로 인한 습기 발생을 억제하고 곰팡이와 같은 유해물질로부터 건축물을 보호하는데 도움이 됩니다. 추가적으로, 수성발제보다 뛰어난 방수성과 내구성을 제공하여 건축 구조물 또는 실내 장식품 등에도 활용됩니다. 따라서, 속건형 유성 발정제를 선택하면 건축물의 내외부 환경을 개선함으로써 안전하고 지속 가능한 건물 건설을 돕습니다.
['벽지에 반점이 생겼을 때, 왜 1년 이내인 경우에만 벽지 아세톤 용제 함침 방법을 사용하고 개선 벽지로 재시공해야 하나요?']
벽지에 반점이 발생한 경우

내단열 방식이 외단열방식보다 상대적으로 더 많은 열교(heat overdischarge)를 유발할 수 있습니다. 이로 인해 내부에 단열재가 설치된 부분과 외부 마감재 사이에 공기층이 형성되어, 이 두 재료간의 온도차로 인해 열교 현상이 쉽게 발생합니다. 이러한 상황에서 열교는 주로 철근 콘크리트 구조 등에서 발생하고, 특히 철골구조나 석고보드 같은 큰 부재를 사용하는 경우에 더욱 자주 발생할 수 있습니다. 따라서 내단열 방식을 고려할 때에는 실내 벽면이나 천장을 재작업 할 시 충분한 환기 및 단열 시스템 구축이 필요합니다. 또한, 적절한 방균 처리 및 곰팡이 방지제를 활용하여 환경을 개선해 주어야 합니다. 만약 내단열 방식으로 교체 작업을 계획 중이라면 전문가의 조언을 구하는 것이 좋습니다. 그리고 만약 기존에 내단열 방식으로 시공했다면, 새로 교체 작업 시에도 동일한 방법으로 재시공해야 합니다. 이렇게 함으로써 보다 효과적인 해결방법을 찾으실 수 있으리라 생각됩니다.
['도배지가 찢어진 경우 터짐 하자가 발생하는 원인과 그에 따른 책임소재는 무엇인가요?', ' 그리고 이를 해결하기 위해 어떤 방법을 사용할 수 있나요?']
도배지 안쪽의 초배지에 인장응력이 작용하여 절단이 생기거나, 벽면에서 생긴 균열로 인해 도배지가 터질 수 있습니다. 이러한 경우 시공자가 작업을 완료하고 나면 추가적인 보수작업으로 해결해야 합니다. 만약 시공자에게 진단 및 조치를 받기 어려운 상황일 때는 시공사나 수리업체에 연락하여 도움을 요청할 필요가 있습니다. 또한, 손상된 벽지나 도배재의 교체 또는 재시공은 필수적으로 고려되어야 하며, 이 과정에서 비용이 많이 들 수도 있으므로 신중함이 요구됩니다. 따라서 적절한 대책 수립이 중요합니다.
도배지 꼬임이 발생한 경우, 합지벽지에 비해 실크 벽지를 추천합니다. 이는 부주의로 인해 꼬임이 발생했거나 공간 내 높은 습도 때문일 가능성이 있는데, 이러한 상황에서는 습기 관리와 적절한 환기가 중요합니다. 따라서 실크 벽지의 접착력과 유지보수 능력

알루미늄 징크는 시간이 지남에 따라 빠르게 경시현상이 나타나며 별도의 방습기능이 없어 옥외조건에서 노출되면 탈색됩니다. 이러한 특성으로 인해 햇빛과 수분으로부터 보호받기를 원하는 장소나 장기간 보관시 변형될 수 있습니다. 또한, 녹 발생 시 방청처리가 제대로 이루어지지 않아 미관상 문제도 고려해야 합니다. 따라서 알루미늄 징크의 경우 부식 방지와 외관 유지를 위해 적절한 관리가 필요합니다.
['겨울철 도배 작업에서 실내 온도를 일정하게 유지해야 하는 이유는 무엇인가요?']
겨울철에 도배작업을 할 때에는 몇 가지 주의할 점이 있습니다. 먼저, 적절한 보온시설을 이용하여 환경적인 영향으로부터 보호해야 합니다. 또한, 충분한 환기와 건조 시간을 주어 습기가 적은 환경을 조성하고, 습도 조절에도 신경써야 합니다. 이러면 도배재료의 접착력과 내구성을 향상시킬 수 있으니 참고해주세요! 추가로, 도배지 두께나 벽면의 경사 등도 고려하여 적절한 방법으로 시공해보시기 바랍니다. 만약 해결되지 않는다면 전문가 상담을 받아 보시길 권장합니다.
['철골구조의 화재 안전 보완을 위한 다양한 방법에는 무엇이 있을까요?', ' 그리고 벽돌구조란 무엇인가요?']
철골 구조물은 매우 높고 무겁기 때문에 화재 발생 시 고온으로 인한 손상과 붕괴가 일어날 수 있습니다. 따라서, 건물 내부에서 화재를 예방하려면 견고하고 튼튼한 벽 및 천장 구조로 설계되어야 하며, 이러한 구조들은 내화성 및 기밀성 또한 고려해야 합니다. 또한 화재 초기에 신속하게 대피할 수 있도록 유도장치와 함께 설치될 필요도 있습니다. 이를 위해 전문가와 협력하여 건축물의 안전성을 높이기 위한 계획 수립이 중요합니다.
벽돌 구조는 건물의 벽체와 기초를 쌓아 만드는데 사용되는 건축 방식입니다. 이 방법은 조적식, 미벽돌쌓기, 블록틀레팅이 혼합된 양식으로 구성됩니다. 이러한 구조는 수평 및 수직 안정성을 확보하여 건축물에 안정감과 안전성을 부여합니다. 또한, 전통적으로 매우 튼튼하고 내구성이 뛰어나며 경제적인 재료로 평가받고 있습

먼저, 물 누수에 의한 곰팡이와 곰팡이 냄새가 발생한 경우에 대해서만 설명드리겠습니다. 1. 수돗물에서 사는 물질들이 원인일 가능성이 큽니다. : 실내 벽면이나 수도관 내부의 곰팡이 또는 배수구로부터 다량의 염분 등이 유입되면 해당 부분을 메우고 있던 수분과 석회암층이 붕괴하여 악취로운 공기 중으로 방출됩니다. 이 때, 곰팡이와 함께 다양한 종류의 박테리아도 번식할 수 있으므로 이러한 환경 조건은 관리 및 예방 측면에서 유의해야 합니다. 2. 건물 내 구조물(창틀, 문틀)에서 오염물질이 유입될 수 있습니다. : 건축물 자체나 주변 환경으로부터 오염물질이 유입된다면, 그 장소에서 발견된 곰팡이와 다른 세균들 또한 쉽게 처리해 줘야 합니다. 3. 물의 온도차가 클수록 곰팡이가 서식하기 좋은 환경이 됩니다. : 고온 다습한 환경에서는 습기가 부족해지므로 곰팡이가 증식하기에 더 용이해집니다. 따라서, 적절한 환기와 정화 시스템을 구축하고 주기적으로 유지하며, 물의 온도 차이를 줄이는 것이 중요합니다. 4. 곰팡이가 존재하면 독성 물질이 생성됩니다. : 곰팡이가 축적됨으로써 유해한 독성분이 발생할 수 있는데, 특히 인장강도가 높은 영역에서는 그러한 위험이 부각됩니다. 그러므로 공간 내에는 균일하게 건조시키고, 필요시 추가적인 보호조치를 취하여 안전을 확보해야 합니다. 5. 누수가 해결되지 않으면 재발할 우려가 있습니다. : 누수가 완전히 제거되지 않았거나 장기간 수리 작업이 이루어지지 않는다면, 누수가 다시 일어날 확률이 높아질 수 있습니다. 때문에 신속하고 철저한 대책이 필요합니다. 6. 곰팡이가 사라지더라도 얼룩이 남을 수 있습니다. : 누수가 제거됐다 하더라도 얼룩이 남아있다면 곰팡이가 다시 생길 수 있습니다. 이런 상황을 방지하려면 얼룩 주위를 깨끗하게 닦아내야 하고, 얼룩이 없어지도록 철저히 관리해야 합니다. 7. 곰팡이가 너무 많아도 문제가 될 수 있습니다. : 과도한 곰팡이는 건물 구조물을 부식시켜 녹슬음 현상을 유발시킬 수 있으며, 결국엔 파괴적인 결과를

내진 설계를 할 때에는 건물의 구조와 재료, 그리고 지진 발생 시 움직임 등을 고려하여 안전성을 평가합니다. 그 후 안전한 설계방식과 규정된 절차대로 재구조화 및 보강을 진행해야 합니다. 먼저 건물 자체의 튼튼함이 중요하며, 이를 위해 내진재로 보강하고 견고한 기초를 구축해야 합니다. 또한 건물 주변이나 지반의 변화나 산사태로부터 보호할 수 있도록 설계자가 필요합니다. 이러한 과정들은 지진으로 인한 피해를 최소화하는데 도움이 됩니다. 따라서 내진 설계시에는 안전성을 확보하기 위해 신중한 접근이 요구됩니다.
내진벽과 같은 구조로 지진이 발생했을 때 건물이나 시설물에 대한 피해를 최소화하기 위해 설치된 벽 구조물입니다. 주로 철근 콘크리트와 같은 무거운 재료로 만들어지며, 높은 설계 및 시공 기준으로 인해 고층 빌딩에서는 필수적으로 사용됩니다. 이러한 내진벽의 장점 중 하나는 내구성이 뛰어나고 시간이 지나도 변형되거나 파손될 가능성이 적다는 것입니다. 또한, 지진 시 충격으로부터 보호하여 건축물 안전성을 높이는 역할을 합니다. 그러나 내진벽은 무겁고 복잡해서 건축 과정에서 많은 비용이 소요된다는 단점이 있습니다. 따라서 추가적인 보강 작업이 필요할 수 있으며, 안전을 위해서는 전문가의 조언을 구하는 것이 좋습니다.
['분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요?', ' 그리고 아파트 도배 평수를 어떻게 계산해야 하나요?']
분말 소화기의 경우, 사용 전에 반드시 질소 가스가 새지 않도록 노즐을 열거나 레버를 눌러서 공기압으로 불을 꺼내야 합니다. 또한, 안전사용을 위해 물이나 알코올 등 가연성 물질로부터 노출되는 것을 피해야 하며, 화재 초기에 신속하게 화염을 제거한 후 환기시켜야 합니다. 이를 통해 안전하고 효과적인 화재 진압이 가능합니다.
아파트 도배평수는 일반적으로 분양 평수*2.5로 설정됩니다. 이는 기존에 살고 있던 거주자들의 크기를 고려하여 산정되며, 새 거주지에서 다시 시작하고자 하는 경우에도 동일한 기준으로 적용됩니다. 따라서 도배 작업

In [16]:
# preds = []
# for t in tqdm(test_df['질문'], total = len(test_df)):
#     result = ask(t)
#     preds.append(result)

In [22]:
preds[:5]

['방청페인트를 사용하려면 몇 가지 특성을 고려해야 합니다. 먼저, 방청페인트는 도장 후 경화되기 전까지 표면이 건조하고 유지될 수 있어야 하며, 이로 인해 도막의 두께가 두꺼워지거나 균일해지지 않을 수 있습니다. 따라서 이러한 특징으로 인해 방청페인트는 주로 외부 환경이나 온도 변화 등에 노출되는 장소에서 사용됩니다. 또한, 방청페인트는 일반적으로 수성 타입의 제품입니다. 물과 같은 용매를 기반으로 만들어지므로 매우 유연하여 쉽게 덧칠할 수 있지만, 내수성 및 내후성이 강하여 특정 조건에서도 잘 견딜 수 있다는 점도 염두에 두시기 바랍니다. 마지막으로, 방청페인트는 철재 구조물 또는 금속 부품의 부식 방지와 보호에도 효과적인 역할을 수행합니다. 이를 통해 건물 내부나 외부의 안전성과 수명을 향상시키는데 도움을 줄 수 있습니다. 원목사이딩의 장점은 가격대가 저렴하고 시공 및 유지보수가 용이하며 자연스러운 나뭇결로 미적인 측면에서도 좋다는 점입니다. 반면에 단점으로는 습기에 취약하여 변형이나 손상될 수 있고, 곰팡이와 같은 해로운 물질로부터 보호되어야 한다는 점이 있습니다. 이러한 이유로 원목사이딩을 선택할 때는 주변 환경과 상황 등을 고려하여 적절한 재료를 선정해야 합니다.',
 '도배지 안쪽의 철착망 형태의 베어링이 오랫동안 사용되면 녹이 슬게 되어, 이염될 수 있습니다. 따라서 이러한 현상을 방지하기 위해서는 적절한 관리가 필요합니다. 먼저, 온도와 습도를 적절히 유지하고 녹이 생긴 부위에 윤활제나 도포재 등을 발라주어야 합니다. 또한, 부식된 부분을 제거하여 청결 상태를 유지해야 하며, 녹이 없도록 교체 작업이나 수리 작업 시 전문가에게 상담받는 것이 좋습니다. 만약 이러한 조치들이 부적절할 경우 곰팡이가 번식할 가능성이 높아질 수도 있으므로 주의해야 합니다. 따라서, 정기적인 점검 및 보수로 건강한 실내 환경 조성에도 신경 써야 합니다.',
 '큐블럭은 일반 벽돌에 비해 높은 비용과 균열이 쉽게 생길 수 있는 점, 그리고 습기로 인해 하자가 발생할 가능

In [18]:
pd.DataFrame({'질문':test_df['질문'],'답변':preds}).to_csv('koalpaca_preprocess_no_shuffle(text).csv', index = False)

### submission

In [19]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [20]:
submit = pd.read_csv('../sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.001155,0.025598,-0.010704,-0.003303,0.098684,0.046992,0.065255,0.060832,-0.012994,...,-0.013456,-0.015402,0.010066,-0.041506,-0.025528,0.049169,0.003093,0.055664,-0.048625,-0.002515
1,TEST_001,-0.018159,0.064750,0.059561,0.030496,0.086919,-0.030323,0.042222,-0.010331,-0.001743,...,-0.024377,-0.050489,0.031540,-0.045971,-0.002270,0.055414,-0.028015,-0.032459,-0.025814,-0.032478
2,TEST_002,0.000829,-0.033895,-0.010009,-0.001491,0.138204,-0.028652,-0.001647,0.001326,0.048849,...,-0.025339,-0.017980,0.067660,0.004005,-0.032145,0.000855,-0.008262,-0.023192,-0.034371,0.062538
3,TEST_003,-0.023976,-0.048553,-0.010154,0.025561,0.096071,-0.042089,-0.066490,-0.058889,-0.004406,...,-0.044676,-0.006103,0.038254,-0.025243,0.012191,0.012569,-0.000032,-0.059508,0.004866,0.045150
4,TEST_004,0.014541,-0.031709,0.001396,-0.010435,0.092970,-0.020742,0.036940,0.025649,-0.064158,...,0.039930,0.000377,0.041894,-0.010946,-0.040530,0.010128,-0.015996,-0.017907,-0.036392,0.019607


In [21]:
submit.to_csv('koalpaca_preprocess(no_shuffle).csv', index = False)
pd.read_csv('koalpaca_preprocess(no_shuffle).csv')

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.001155,0.025598,-0.010704,-0.003303,0.098684,0.046992,0.065255,0.060832,-0.012994,...,-0.013456,-0.015402,0.010066,-0.041506,-0.025528,0.049169,0.003093,0.055664,-0.048625,-0.002515
1,TEST_001,-0.018159,0.064750,0.059561,0.030496,0.086919,-0.030323,0.042222,-0.010331,-0.001743,...,-0.024377,-0.050489,0.031540,-0.045971,-0.002270,0.055414,-0.028015,-0.032459,-0.025814,-0.032478
2,TEST_002,0.000829,-0.033895,-0.010009,-0.001491,0.138204,-0.028652,-0.001647,0.001326,0.048849,...,-0.025339,-0.017980,0.067660,0.004005,-0.032145,0.000855,-0.008262,-0.023192,-0.034370,0.062538
3,TEST_003,-0.023976,-0.048553,-0.010154,0.025561,0.096071,-0.042089,-0.066490,-0.058889,-0.004406,...,-0.044676,-0.006103,0.038254,-0.025243,0.012191,0.012569,-0.000032,-0.059508,0.004866,0.045150
4,TEST_004,0.014541,-0.031709,0.001396,-0.010435,0.092970,-0.020742,0.036940,0.025649,-0.064158,...,0.039930,0.000377,0.041894,-0.010946,-0.040530,0.010128,-0.015996,-0.017907,-0.036392,0.019607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,-0.005241,-0.018439,0.009537,-0.006639,0.094256,-0.003721,0.018576,0.072640,0.038371,...,-0.002010,-0.007091,0.059977,-0.039359,0.046067,0.003739,0.047140,-0.057896,0.001016,0.055382
126,TEST_126,0.049239,-0.059101,-0.040873,-0.002245,0.113182,0.007454,0.030288,0.017839,0.029624,...,-0.043569,-0.031050,0.032971,-0.036742,0.024101,0.041854,-0.009481,-0.000269,-0.001276,0.037016
127,TEST_127,-0.030462,0.014475,0.010788,0.003386,0.088162,-0.021904,0.006779,-0.029387,-0.054368,...,-0.033235,-0.061943,0.020223,-0.018466,-0.039802,0.068571,0.041406,0.028161,-0.048241,-0.024022
128,TEST_128,0.049930,0.003881,-0.062978,0.030856,0.128134,-0.082986,0.009560,0.016935,-0.044622,...,-0.039245,0.014632,0.010541,0.006702,-0.033136,-0.007847,-0.017966,-0.041385,-0.001246,-0.014411
